In [1]:
# Import Required Libraries
import os
import pandas as pd
from datetime import datetime, timedelta
import random
import json
import findspark

findspark.init()

# PySpark imports
try:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import *
    from pyspark.sql.types import *
    pyspark_available = True
    print("PySpark is available!")
except ImportError:
    print("PySpark not found. Please install with: pip install pyspark")
    pyspark_available = False

PySpark is available!


In [2]:
if pyspark_available:
    # Create SparkSession with custom configuration
    spark = SparkSession.builder \
        .appName("PySpark") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.driver.memory", "2g") \
        .config("spark.executor.memory", "1g") \
        .getOrCreate()
    
    # Set log level to reduce verbose output
    spark.sparkContext.setLogLevel("WARN")
    
    print("✓ SparkSession created successfully!")
    print(f"Spark Version: {spark.version}")
    print(f"Application Name: {spark.sparkContext.appName}")
    print(f"Master: {spark.sparkContext.master}")
    
    # Check available cores and memory
    print(f"Default Parallelism: {spark.sparkContext.defaultParallelism}")
    
else:
    print("Cannot proceed without PySpark. Please install PySpark first.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/27 16:14:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ SparkSession created successfully!
Spark Version: 3.5.0
Application Name: PySpark
Master: local[*]
Default Parallelism: 4


In [ ]:
import time
import json

if pyspark_available:
    print("=== Bronze Layer: Raw Data Ingestion ===")

    with open("./data/students_no_scores.json/part-00000-3615aed5-2901-4b90-81af-4caacb7dba9a-c000.json", "r") as f:
        lines = f.read().splitlines()

    raw_rdd = spark.sparkContext.parallelize(lines)

    def parse_json_safe(json_str):
        try:
            data = json.loads(json_str)
            data['_ingestion_timestamp'] = time.time()
            data['_source'] = 'file'
            data['_status'] = 'valid'
            return data
        except:
            return {
                '_raw_data': json_str,
                '_ingestion_timestamp': time.time(),
                '_source': 'file',
                '_status': 'parse_error'
            }
        
    bronze_rdd = raw_rdd.map(parse_json_safe)
    bronze_data = bronze_rdd.collect()

    # Saving bronze data to a json file
    json_file_path = "./data/bronze/students_without_exam.json"
    with open(json_file_path, "w", encoding="utf-8") as f:
        json.dump(bronze_data, f, indent=2)  


=== Bronze Layer: Raw Data Ingestion ===


25/11/27 16:14:49 WARN TaskSetManager: Stage 0 contains a task of very large size (1011 KiB). The maximum recommended task size is 1000 KiB.
